# Mapeamento de patentes

## Bibliotecas

In [1]:
import pandas as pd
import numpy as np

## Limpeza (Manual)

In [2]:
# Exporte o DataFrame para o arquivo Excel
df = pd.read_excel('data_patentes_excel.xlsx')
# Crie um DataFrame com a coluna 'ID'
df.insert(0, 'ID', ["ID{:05d}".format(i) for i in range(1, len(df) + 1)])
df.head()

,ID,data_publicacao,diretoria,numero_revista,codigo,titulo,numero,data_deposito,data_prioridade,cod_prioridade,classif_internacion,classif_nacion,titulo_inid,titular_nome,titular_uf,titular_pais,inventor_nome
0,ID00001,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2013 016644-8,27/06/2013,NaN,NaN,B29B 17/00; B29C 43/00; B29K 31/00,B29B 17/00; B29C 43/00,APERFEIÇOAMENTO INTRODUZIDO EM PROCESSO DE REC...,ACM EL ORRA COLCHÕES ME,SP,BR,ABDO CARIM MOUMTEZ EL ORRA
1,ID00002,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2014 001105-6,16/01/2014,18/01/2013,TW102201214,F16C 11/04; E05F 1/12,F16C 11/04; E05F 1/1215,DOBRADIÇA PARA CONJUNTO DE CAPAS PROTETORAS,"KEYARROW (TAIWAN) CO., LTD.",NaN,TW,"FU-YI CHEN; CHI, LI-LING"
2,ID00003,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 004929-9,13/03/2017,NaN,NaN,H01Q 13/10; H01Q 1/24,H01Q 13/106; H01Q 1/241,"UMA NOVA ANTENA DE MICROFITA TRI-BANDA, DE TAM...",UNIVERSIDADE FEDERAL DO CEARÁ,CE,BR,ANTONIO SERGIO BEZERRA SOMBRA; RONALDO GLAUBER...
3,ID00004,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 024659-0,17/11/2017,NaN,NaN,G01F 1/58; H04W 4/12,G01F 1/58; H04W 4/12,DISPOSITIVO DE MÚLTIPLOS MÓDULOS COM OTIMIZAÇÃ...,CID GRANJA DOS SANTOS,NaN,BR,CID GRANJA DOS SANTOS
4,ID00005,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 026178-6,05/12/2017,NaN,NaN,H04L 29/06; H04L 29/12,H04L 67/42; H04L 61/1558; H04L 29/12141,BEHAVIOR DEFINITION MODEL FOR INTERNET OF THIN...,FUNDAÇÃO UNIVERSIDADE DO AMAZONAS,AM,BR,EDUARDO JAMES PEREIRA SOUTO; NAMEDIN PEREIRA T...


In [3]:
df_responsavel = df[['ID', 'titular_nome']]
df_responsavel.head()
df_responsavel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24714 entries, 0 to 24713
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            24714 non-null  object
 1   titular_nome  24714 non-null  object
dtypes: object(2)
memory usage: 386.3+ KB


In [4]:
num_elementos = df_responsavel['titular_nome'].copy()
#print(num_elementos)
num_elementos.apply(lambda x: len(x.split(';'))).max()

35

In [5]:
responsavel = ["responsavel{:02d}".format(i) for i in range(1, 36)]

df_responsavel.loc[:, responsavel] = df_responsavel['titular_nome'].str.split(';', expand=True).copy()

In [6]:
responsavel_melted = pd.melt(df_responsavel, 
                    id_vars=['ID'], 
                    value_vars=["responsavel{:02d}".format(i) for i in range(1, 36)], 
                    var_name='ordem_respons', 
                    value_name='responsavel')

responsavel_melted = responsavel_melted.dropna(subset= ['responsavel'])
responsavel_melted = responsavel_melted.sort_values('ID')

def criar_nova_variavel(row):
    if row['ordem_respons'] == 'responsavel01':
        return 'Titular'
    else:
        return 'Parceiro'
    
responsavel_melted.loc[:, 'parceria'] = responsavel_melted.apply(criar_nova_variavel, axis=1).copy()
responsavel_melted['responsavel'] = responsavel_melted['responsavel'].apply(lambda x: str(x).strip())

responsavel_melted.head(10)


,ID,ordem_respons,responsavel,parceria


In [7]:
tab = responsavel_melted.groupby('ID').count()
n = tab.groupby('responsavel').count().reset_index()
n['porcentagem'] = round((n['ordem_respons'] / n['ordem_respons'].sum()) * 100,2)

print(n)

Empty DataFrame
Columns: [responsavel, ordem_respons, parceria, porcentagem]
Index: []


In [8]:
responsavel_melted.to_excel('responsavel_melted.xlsx', sheet_name='Parcerias', index=False)

In [9]:
responsavel_melted_revisado = pd.read_excel('responsavel_melted_revisado.xlsx')
responsavel_melted_revisado.head(10)

,ID,ordem_respons,responsavel,parceria
0,ID00001,titular01,ACM EL ORRA COLCHÕES ME,Titular
1,ID00002,titular01,"KEYARROW (TAIWAN) CO., LTD.",Titular
2,ID00003,titular01,UNIVERSIDADE FEDERAL DO CEARÁ,Titular
3,ID00004,titular01,CID GRANJA DOS SANTOS,Titular
4,ID00005,titular01,FUNDAÇÃO UNIVERSIDADE DO AMAZONAS,Titular
5,ID00006,titular01,MISSÃO SALESIANA DE MATO GROSSO UNIVERSIDADE C...,Titular
6,ID00007,titular01,GRENDENE S.A.,Titular
7,ID00008,titular01,RENTBRELLA S.A.,Titular
8,ID00009,titular01,FUNDAÇÃO UNIVERSIDADE ESTADUAL DO CEARÁ,Titular
9,ID00010,titular01,MARCUS CEZAR KLAUTAU BONNA,Titular


In [10]:
tab = responsavel_melted_revisado.groupby('responsavel').agg(n=('responsavel', 'size')).reset_index()
tab['porcentagem'] = round((tab['n'] / tab['n'].sum()) * 100,2)
tab = tab.sort_values(by='n', ascending=False)
print(tab)

                                             responsavel    n  porcentagem
3003                                     DEERE & COMPANY  637         2.18
12589                    UNIVERSIDADE FEDERAL DA PARAÍBA  295         1.01
12604               UNIVERSIDADE FEDERAL DE MINAS GERAIS  292         1.00
12592             UNIVERSIDADE FEDERAL DE CAMPINA GRANDE  279         0.95
10084               PETRÓLEO BRASILEIRO S.A. - PETROBRAS  264         0.90
...                                                  ...  ...          ...
5035                         GETÚLIO WASHINGTON CASTILHO    1         0.00
5036                                  GEZIEL COSTA GOMES    1         0.00
5037                          GEZIEL FERREIRA HERMÓGENES    1         0.00
5038   GFD ADMINISTRAÇÃO PATRIMONIAL E PARTICIPAÇÕES ...    1         0.00
13425  ÓTIMA INDÚSTRIA, COMÉRCIO, IMPORTAÇÃO E EXPORT...    1         0.00

[13426 rows x 3 columns]


## Limpeza (Automatizada)

In [11]:
data = pd.read_excel('data_patentes_excel.xlsx')
data.insert(0, 'ID', ["ID{:05d}".format(i) for i in range(1, len(data) + 1)])
data[responsavel] = data['titular_nome'].str.split(';', expand=True)
data.head(5)

,ID,data_publicacao,diretoria,numero_revista,codigo,titulo,numero,data_deposito,data_prioridade,cod_prioridade,...,responsavel26,responsavel27,responsavel28,responsavel29,responsavel30,responsavel31,responsavel32,responsavel33,responsavel34,responsavel35
0,ID00001,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2013 016644-8,27/06/2013,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1,ID00002,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2014 001105-6,16/01/2014,18/01/2013,TW102201214,...,None,None,None,None,None,None,None,None,None,None
2,ID00003,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 004929-9,13/03/2017,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
3,ID00004,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 024659-0,17/11/2017,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
4,ID00005,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 026178-6,05/12/2017,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24714 entries, 0 to 24713
Data columns (total 52 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   24714 non-null  object 
 1   data_publicacao      24714 non-null  object 
 2   diretoria            24714 non-null  object 
 3   numero_revista       24714 non-null  int64  
 4   codigo               24714 non-null  float64
 5   titulo               24714 non-null  object 
 6   numero               24714 non-null  object 
 7   data_deposito        24714 non-null  object 
 8   data_prioridade      5052 non-null   object 
 9   cod_prioridade       5052 non-null   object 
 10  classif_internacion  24223 non-null  object 
 11  classif_nacion       17269 non-null  object 
 12  titulo_inid          24714 non-null  object 
 13  titular_nome         24714 non-null  object 
 14  titular_uf           19438 non-null  object 
 15  titular_pais         24714 non-null 

In [13]:
data_melted = pd.melt(data, 
                    id_vars=['ID'], 
                    value_vars=["responsavel{:02d}".format(i) for i in range(1, 36)], 
                    var_name='ordem_respons', 
                    value_name='responsavel').copy()

data_melted = data_melted.dropna(subset= ['responsavel'])
data_melted = data_melted.sort_values('ID')

def criar_nova_variavel(row):
    if row['ordem_respons'] == 'responsavel01':
        return 'Titular'
    else:
        return 'Parceiro'
    
data_melted['parceria'] = data_melted.apply(criar_nova_variavel, axis=1)
data_melted['responsavel'] = data_melted['responsavel'].str.strip()
data_melted['responsavel'] = data_melted['responsavel'].replace("  ", " ")
data_melted['responsavel'] = data_melted['responsavel'].replace("  ", " ")
data_melted.head(10)


,ID,ordem_respons,responsavel,parceria
0,ID00001,responsavel01,ACM EL ORRA COLCHÕES ME,Titular
1,ID00002,responsavel01,"KEYARROW (TAIWAN) CO., LTD.",Titular
2,ID00003,responsavel01,UNIVERSIDADE FEDERAL DO CEARÁ,Titular
3,ID00004,responsavel01,CID GRANJA DOS SANTOS,Titular
4,ID00005,responsavel01,FUNDAÇÃO UNIVERSIDADE DO AMAZONAS,Titular
5,ID00006,responsavel01,MISSÃO SALESIANA DE MATO GROSSO UNIVERSIDADE C...,Titular
6,ID00007,responsavel01,GRENDENE S.A.,Titular
7,ID00008,responsavel01,RENTBRELLA S.A.,Titular
8,ID00009,responsavel01,FUNDAÇÃO UNIVERSIDADE ESTADUAL DO CEARÁ - FUNECE,Titular
9,ID00010,responsavel01,MARCUS CEZAR KLAUTAU BONNA,Titular


## Filtro das IFES

In [14]:
chave = r'FEDERAL|FEDEERAL|UNIFEI|UNIVERSIDADE DE BRASILIA|UNIVERSIDADE DE BRASÍLIA|UNIVERSIDADE DA INTEGRAÇÃO INTERNACIONAL|UNIVERSIDADE DA INTEGRACAO INTERNACIONAL'
filtro = data_melted['responsavel'].str.contains(chave, case=False, regex=True) & ~data_melted['responsavel'].str.contains('FEDERAL CRED', case=False, regex=True)
federais_melted = data_melted[filtro].copy()
federais_melted['ifes'] = 'Sim'
federais_melted.head()

,ID,ordem_respons,responsavel,parceria,ifes
2,ID00003,responsavel01,UNIVERSIDADE FEDERAL DO CEARÁ,Titular,Sim
24724,ID00011,responsavel02,UNIVERSIDADE FEDERAL DE PERNAMBUCO,Parceiro,Sim
10,ID00011,responsavel01,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Titular,Sim
14,ID00015,responsavel01,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Titular,Sim
23,ID00024,responsavel01,UNIVERSIDADE FEDERAL DE UBERLÂNDIA,Titular,Sim


In [15]:
tab = federais_melted.groupby('responsavel').agg(n=('responsavel', 'size')).reset_index()
tab = tab.sort_values(by='n', ascending=False)
print(tab["responsavel"])

174                 UNIVERSIDADE FEDERAL DE MINAS GERAIS
156          UNIVERSIDADE FEDERAL DE CAMPINA GRANDE - PB
141                      UNIVERSIDADE FEDERAL DA PARAIBA
183                   UNIVERSIDADE FEDERAL DE PERNAMBUCO
205                   UNIVERSIDADE FEDERAL DE UBERLÂNDIA
                             ...                        
125           UNIVERSIDADE  FEDERAL DE OURO PRETO - UFOP
126    UNIVERSIDADE DA INTEGRAÇÃO INTERNACIONAL DA LU...
128                       UNIVERSIDADE DE BRASILIA - UNB
130          UNIVERSIDADE FEDEERAL DE SANTA MARIA - UFSM
288                        Universidade Federal do Piauí
Name: responsavel, Length: 289, dtype: object


In [16]:
#tab["responsavel"].to_excel('filtro.xlsx', sheet_name='Federais', index=False)

In [17]:
#Verificar dados
non_ifes_melted = data_melted[~filtro]

non_ifes_melted.head(15)


,ID,ordem_respons,responsavel,parceria
0,ID00001,responsavel01,ACM EL ORRA COLCHÕES ME,Titular
1,ID00002,responsavel01,"KEYARROW (TAIWAN) CO., LTD.",Titular
3,ID00004,responsavel01,CID GRANJA DOS SANTOS,Titular
4,ID00005,responsavel01,FUNDAÇÃO UNIVERSIDADE DO AMAZONAS,Titular
5,ID00006,responsavel01,MISSÃO SALESIANA DE MATO GROSSO UNIVERSIDADE C...,Titular
6,ID00007,responsavel01,GRENDENE S.A.,Titular
7,ID00008,responsavel01,RENTBRELLA S.A.,Titular
8,ID00009,responsavel01,FUNDAÇÃO UNIVERSIDADE ESTADUAL DO CEARÁ - FUNECE,Titular
9,ID00010,responsavel01,MARCUS CEZAR KLAUTAU BONNA,Titular
49438,ID00011,responsavel03,ASSOCIAÇÃO INSTITUTO DE TECNOLOGIA DE PERNAMBU...,Parceiro


In [18]:
#CRIAR NOVA COLUNA COPIANDO A COLUNA DO NOME DO TITULAR PARA ABREVIAR/AGRUPAR
federais_melted['ifes_abrev'] = federais_melted['responsavel'].copy()
federais_melted.head()


,ID,ordem_respons,responsavel,parceria,ifes,ifes_abrev
2,ID00003,responsavel01,UNIVERSIDADE FEDERAL DO CEARÁ,Titular,Sim,UNIVERSIDADE FEDERAL DO CEARÁ
24724,ID00011,responsavel02,UNIVERSIDADE FEDERAL DE PERNAMBUCO,Parceiro,Sim,UNIVERSIDADE FEDERAL DE PERNAMBUCO
10,ID00011,responsavel01,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Titular,Sim,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO..."
14,ID00015,responsavel01,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Titular,Sim,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO..."
23,ID00024,responsavel01,UNIVERSIDADE FEDERAL DE UBERLÂNDIA,Titular,Sim,UNIVERSIDADE FEDERAL DE UBERLÂNDIA


## Dicionário de correspondências

In [19]:
#CRIAR DICIONARIO DE CORRESPONDENCIAS E APLICAR A COLUNA DE NOME DO TITULAR ABREVIADO

correspondencias = {
    ': UNIVERSIDADE FEDERAL DE SANTA MARIA - UFSM' : 'UFSM',
    'A FUNDAÇÃO UNIVERSIDADE FEDERAL DE SÃO CARLOS - UFSCAR' : 'UFSCAR',
    'A UNIVERSIDADE FEDERAL DE GOIÁS - UFG' : 'UFG',
    'CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA CELSO SUCKOW DA FONSECA - CEFET/RJ' : 'CEFET-RJ',
    'CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA DE MINAS GERAIS' : 'CEFET-MG',
    'CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA DE MINAS GERAIS - CEFET/MG' : 'CEFET-MG',
    'CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA DE MINAS GERAIS ? CEFET/MG' : 'CEFET-MG',
    'EMPRESA BRASILEIRA DE SERVIÇOS HOSPITALARES - EBSERH, -FILIAL HOSPITAL UNIVERSITÁRIO PROFESSOR POLYDORO ERNANI DE SÃO THIAGO, DA UNIVERSIDADE FEDERAL DE SANTA CATARINA (HU-UFSC)' : 'UFSC',
    'FUNDACAO UNIVERSIDADE DE BRASILIA' : 'UNB',
    'FUNDAÇÃO UNIVERSIDADE DE BRASILIA' : 'UNB',
    'FUNDAÇÃO UNIVERSIDADE DE BRASÍLIA' : 'UNB',
    'FUNDAÇÃO UNIVERSIDADE DE BRASILIA - FUB' : 'UNB',
    'FUNDAÇÃO UNIVERSIDADE DE BRASÍLIA - FUB' : 'UNB',
    'FUNDAÇÃO UNIVERSIDADE DE BRASÍLIA (FUB)' : 'UNB',
    'FUNDAÇÃO UNIVERSIDADE DE BRASILIA/CENTRO DE APOIO AO DESENVOLVIMENTO TECNOLÓGICO' : 'UNB',
    'FUNDACAO UNIVERSIDADE FEDERAL DE CIENCIAS DA SAUDE DE PORTO ALEGRE' : 'UFCSPA',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DE MATO GROSSO' : 'UFMT',
    'FUNDACAO UNIVERSIDADE FEDERAL DE MATO GROSSO DO SUL' : 'UFMS',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DE MATO GROSSO DO SUL' : 'UFMS',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DE MATO GROSSO DO SUL - (UFMS)' : 'UFMS',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DE PELOTAS' : 'UFPEL',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DE PELOTAS - UFPEL' : 'UFPEL',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DE SÃO CARLOS' : 'UFSCAR',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DE SÃO CARLOS - UFSCAR' : 'UFSCAR',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DE SÃO CARLOS ? UFSCAR' : 'UFSCAR',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DE SÃO CARLOS ? UFSCAR,' : 'UFSCAR',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DO ABC' : 'UFABC',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DO ABC - UFABC' : 'UFABC',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DO ABC- UFABC' : 'UFABC',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DO ACRE' : 'UFAC',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DO AMAPÁ' : 'UNIFAP',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DO MATO GROSSO DO SUL' : 'UFMS',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DO PAMPA' : 'UNIPAMPA',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DO PIAUÍ' : 'UFPI',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DO PIAUÍ - UFPI' : 'UFPI',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DO RIO GRANDE' : 'FURG',
    'FUNDAÇÃO UNIVERSIDADE FEDERAL DO TOCANTINS' : 'UFT',
    'FUNDACAO UNIVERSIDADE FEDERAL DO VALE DO SAO FRANCISCO' : 'UNIVASF',
    'IFTM - INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO TRIÂNGULO MINEIRO' : 'IFTM',
    'INSTITUTO FEDERAL CATARINENSE' : 'IFC',
    'INSTITUTO FEDERAL DA PARAÍBA' : 'IFPB',
    'INSTITUTO FEDERAL DE  EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE MINAS GERAIS' : 'IFMG',
    'INSTITUTO FEDERAL DE ALAGOAS' : 'IFAL',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIENCIA E TECNOLOGIA DA BAHIA' : 'IFPB',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOLOGIA DA PARAÍBA' : 'IFPB',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOLOGIA DA PARAÍBA - IFPB' : 'IFPB',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIENCIA E TECNOLOGIA DE MINAS GERAIS' : 'IFMG',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOLOGIA DO AMAPÁ' : 'IFAP',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOLOGIA DO AMAZONAS' : 'IFAM',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOLOGIA DO ESPIRITO SANTO' : 'IFES',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOLOGIA DO ESPÍRITO SANTO' : 'IFES',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOLOGIA DO ESPÍRITO SANTO - IFES' : 'IFES',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOLOGIA DO MARANHÃO' : 'IFMA',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOLOGIA DO MARANHÃO - IFMA' : 'IFMA',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOLOGIA DO PARÁ' : 'IFPA',
    'INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOLOGIA DO SERTÃO PERNAMBUCANO' : 'IFSertãoPE',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA' : 'IF',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA  GOIANO' : 'IFGOIANO',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA BAIANO' : 'IFBAIANO',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA BAIANO - IFBAIANO' : 'IFBAIANO',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA CATARINENSE' : 'IFC',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA CATARINENSE (IFC)' : 'IFC',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DA BAHIA' : 'IFBA',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DA BAHIA - IFBA' : 'IFBA',
    'INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNOLOGIA DA PARAIBA' : 'IFPB',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DA PARAÍBA' : 'IFPB',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DA PARAÍBA - CAMPUS CATOLÉ DO ROCHA' : 'IFPB',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DA PARAÍBA - IFPB' : 'IFPB',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE  PERNAMBUCO (IFPE)' : 'IFPE',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE ALAGOAS' : 'IFAL',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE ALAGOAS - IFAL' : 'IFAL',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE BRASÍLIA' : 'IFDF',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE GOIÁS' : 'IFG',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE GOIÁS-IFG' : 'IFG',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE MATO GROSSO' : 'IFMT',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE MATO GROSSO DO SUL' : 'IFMS',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE MINAS GERAIS' : 'IFMG',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE MINAS GERAIS ? IFMG' : 'IFMG',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE PERNAMBUCO' : 'IFPE',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE RONDÔNIA' : 'IFRO',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE RORAIMA' : 'IFRR',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SANTA CATARINA' : 'IFSC',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO' : 'IFSP',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIENCIA E TECNOLOGIA DO  PARÁ' : 'IFPA',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO CEARÁ' : 'IFCE',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO CEARÁ - IFCE' : 'IFCE',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO ESPÍRITO SANTO' : 'IFES',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO MARANHÃO' : 'IFMA',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO PARÁ' : 'IFPA',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO PARANÁ' : 'IFPR',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO PARANÁ ? IFPR' : 'IFPR',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO PIAUÍ' : 'IFPI',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO RIO DE JANEIRO' : 'IFRJ',
    'INSTITUTO FEDERAL DE EDUCACAO, CIÊNCIA E TECNOLOGIA DO RIO DE JANEIRO - IFRJ' : 'IFRJ',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO RIO DE JANEIRO - IFRJ' : 'IFRJ',
    'INSTITUTO FEDERAL DE EDUCACAO, CIÊNCIA E TECNOLOGIA DO RIO DE JANEIRO - IFRJ,' : 'IFRJ',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO RIO DE JANEIRO (IFRJ)' : 'IFRJ',
    'INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNOLOGIA DO RIO GRANDE DO NORTE' : 'IFRN',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO RIO GRANDE DO NORTE' : 'IFRN',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO RIO GRANDE DO NORTE - IFRN' : 'IFRN',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO RIO GRANDE DO SUL' : 'IFRS',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO RIO GRANDE DO SUL - IFRS' : 'IFRS',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO SERTÃO PERNAMBUCANO' : 'IFSERTAOPE',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO SUDESTE DE MINAS' : 'IFSEMG',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO SUL DE MINAS GERAIS - IFSULDEMINAS' : 'IFSULDEMINAS',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO TOCANTINS' : 'IFTO',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO TRIÂNGULO MINEIRO' : 'IFTM',
    'INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNOLOGIA FARROUPILHA' : 'IFFARROUPILHA',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA FLUMINENSE' : 'IFFLUMINENSE',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA FLUMINENSE - IFFLUMINENSE' : 'IFFLUMINENSE',
    'INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA GOIANO' : 'IFGOIANO',
    'INSTITUTO FEDERAL DE EDUCAÇÃO,CIÊNCIA E TECNOLOGIA DA PARAÍBA' : 'IFPB',
    'INSTITUTO FEDERAL DE EDUCAÇÃO,CIENCIA E TECNOLOGIA DO ACRE' : 'IFAC',
    'INSTITUTO FEDERAL DE EDUCAÇÃO,CIÊNCIA E TECNOLOGIA DO PARÁ' : 'IFPA',
    'INSTITUTO FEDERAL DE MATO GROSSO - IFMT' : 'IFMT',
    'INSTITUTO FEDERAL DE SANTA CATARINA' : 'IFSC',
    'INSTITUTO FEDERAL DO NORTE DE MINAS GERAIS' : 'IFNORTEDEMINAS',
    'INSTITUTO FEDERAL DO PARANÁ' : 'IFPR',
    'INSTITUTO FEDERAL DO PARANÁ - IFPR' : 'IFPR',
    'INSTITUTO FEDERAL DO RIO GRANDE DO NORTE' : 'IFRN',
    'INSTITUTO FEDERAL DO TOCANTINS' : 'IFTO',
    'INSTITUTO FEDERAL FARROUPILHA - IFFAR' : 'IFFAR',
    'INSTITUTO FEDERAL SUL-RIO-GRANDENSE' : 'IFSUL',
    'UFPA - UNIVERSIDADE FEDERAL DO PARÁ' : 'UFPA',
    'UNIFEI - UNIVERSIDADE FEDERAL DE ITAJUBÁ' : 'UNIFEI',
    'UNIVERSIDADE  FEDERAL DE OURO PRETO - UFOP' : 'UFOP',
    'UNIVERSIDADE DA INTEGRAÇÃO INTERNACIONAL DA LUSOFONIA AFRO-BRASILEIRA - UNILAB' : 'UNILAB',
    'UNIVERSIDADE DA INTEGRAÇÃO INTERNACIONAL DA LUSOFONIA AFRO-BRASILEIRA (UNILAB)' : 'UNILAB',
    'UNIVERSIDADE DE BRASÍLIA' : 'UNB',
    'UNIVERSIDADE DE BRASILIA - UNB' : 'UNB',
    'UNIVERSIDADE FEDEERAL DE SANTA MARIA - UFSM' : 'UFSM',
    'UNIVERSIDADE FEDERAL 00 RIO DE JANEIRO -UFRJ' : 'UFRJ',
    'UNIVERSIDADE FEDERAL DA BAHIA' : 'UFBA',
    'UNIVERSIDADE FEDERAL DA BAHIA - UFBA' : 'UFBA',
    'UNIVERSIDADE FEDERAL DA FRONTEIRA SUL, UFFS' : 'UFFS',
    'UNIVERSIDADE FEDERAL DA GRANDE DOURADOS' : 'UFGD',
    'UNIVERSIDADE FEDERAL DA GRANDE DOURADOS - UFGD' : 'UFGD',
    'UNIVERSIDADE FEDERAL DA INTEGRAÇÃO  LATINO-AMERICANA - UNILA' : 'UNILA',
    'UNIVERSIDADE FEDERAL DA INTEGRAÇÃO LATINO-AMERICANA' : 'UNILA',
    'UNIVERSIDADE FEDERAL DA INTEGRAÇÃO LATINOAMERICANA - UNILA' : 'UNILA',
    'UNIVERSIDADE FEDERAL DA INTEGRAÇÃO LATINO-AMERICANA - UNILA' : 'UNILA',
    'UNIVERSIDADE FEDERAL DA PARAIBA' : 'UFPB',
    'UNIVERSIDADE FEDERAL DA PARAÍBA' : 'UFPB',
    'UNIVERSIDADE FEDERAL DA PARAIBA - UFPB' : 'UFPB',
    'UNIVERSIDADE FEDERAL DA PARAÍBA - UFPB' : 'UFPB',
    'UNIVERSIDADE FEDERAL DE  ALFENAS - UNIFAL-MG' : 'UNIFAL-MG',
    'UNIVERSIDADE FEDERAL DE  LAVRAS - UFLA' : 'UFLA',
    'UNIVERSIDADE FEDERAL DE  SÃO JOÃO DEL REI - UFSJ' : 'UFSJ',
    'UNIVERSIDADE FEDERAL DE ALAGOAS' : 'UFAL',
    'UNIVERSIDADE FEDERAL DE ALAGOAS - UFAL' : 'UFAL',
    'UNIVERSIDADE FEDERAL DE ALFENAS' : 'UNIFAL-MG',
    'UNIVERSIDADE FEDERAL DE ALFENAS - UNIFAL' : 'UNIFAL-MG',
    'UNIVERSIDADE FEDERAL DE ALFENAS - UNIFAL-MG' : 'UNIFAL-MG',
    'UNIVERSIDADE FEDERAL DE ALFENAS -UNIFAL' : 'UNIFAL-MG',
    'UNIVERSIDADE FEDERAL DE CAMPINA GRANDE' : 'UFCG',
    'UNIVERSIDADE FEDERAL DE CAMPINA GRANDE - PB' : 'UFCG',
    'UNIVERSIDADE FEDERAL DE CAMPINA GRANDE - UFCG' : 'UFCG',
    'UNIVERSIDADE FEDERAL DE CAMPINA GRANDE (UFCG)' : 'UFCG',
    'UNIVERSIDADE FEDERAL DE GOIAS' : 'UFG',
    'UNIVERSIDADE FEDERAL DE GOIÁS' : 'UFG',
    'UNIVERSIDADE FEDERAL DE GOIÁS - UFG' : 'UFG',
    'UNIVERSIDADE FEDERAL DE GOIÁS-UFG' : 'UFG',
    'UNIVERSIDADE FEDERAL DE ITAJUBÁ' : 'UNIFEI',
    'UNIVERSIDADE FEDERAL DE ITAJUBÁ - UNIFEI' : 'UNIFEI',
    'UNIVERSIDADE FEDERAL DE ITAJUBA- UNIFEI' : 'UNIFEI',
    'UNIVERSIDADE FEDERAL DE JATAI' : 'UFJ',
    'UNIVERSIDADE FEDERAL DE JUIZ  DE FORA - UFJF' : 'UFJF',
    'UNIVERSIDADE FEDERAL DE JUIZ DE FORA' : 'UFJF',
    'UNIVERSIDADE FEDERAL DE JUIZ DE FORA - UFJF' : 'UFJF',
    'UNIVERSIDADE FEDERAL DE LAVRAS' : 'UFLA',
    'UNIVERSIDADE FEDERAL DE LAVRAS - UFLA' : 'UFLA',
    'UNIVERSIDADE FEDERAL DE LAVRAS,UNIVERSIDADE FEDERAL DE LAVRAS - UFLA' : 'UFLA',
    'UNIVERSIDADE FEDERAL DE MATO GROSSO' : 'UFMT',
    'UNIVERSIDADE FEDERAL DE MATO GROSSO DO SUL' : 'UFMS',
    'UNIVERSIDADE FEDERAL DE MINAS GERAIS' : 'UFMG',
    'UNIVERSIDADE FEDERAL DE MINAS GERAIS - UFMG' : 'UFMG',
    'UNIVERSIDADE FEDERAL DE MINAS GERAIS ? UFMG' : 'UFMG',
    'UNIVERSIDADE FEDERAL DE MINAS GERAIS-UFMG' : 'UFMG',
    'UNIVERSIDADE FEDERAL DE OURO PRETO' : 'UFOP',
    'UNIVERSIDADE FEDERAL DE OURO PRETO  - UFOP' : 'UFOP',
    'UNIVERSIDADE FEDERAL DE OURO PRETO - UFOP' : 'UFOP',
    'UNIVERSIDADE FEDERAL DE OURO PRETO ? UFOP' : 'UFOP',
    'UNIVERSIDADE FEDERAL DE PELOTAS' : 'UFPEL',
    'UNIVERSIDADE FEDERAL DE PERNAMBUCO' : 'UFPE',
    'UNIVERSIDADE FEDERAL DE PERNAMBUCO - UFPE' : 'UFPE',
    'UNIVERSIDADE FEDERAL DE PERNAMBUCO -UFPE' : 'UFPE',
    'UNIVERSIDADE FEDERAL DE RONDONOPOLIS' : 'UFR',
    'UNIVERSIDADE FEDERAL DE RORAIMA' : 'UFRR',
    'UNIVERSIDADE FEDERAL DE SANTA CATARINA' : 'UFSC',
    'Universidade Federal de Santa Catarina' : 'UFSC',
    'UNIVERSIDADE FEDERAL DE SANTA CATARINA - UFSC' : 'UFSC',
    'UNIVERSIDADE FEDERAL DE SANTA MARIA' : 'UFSM',
    'UNIVERSIDADE FEDERAL DE SANTA MARIA - UFSM' : 'UFSM',
    'UNIVERSIDADE FEDERAL DE SANTA MARIA ? UFSM' : 'UFSM',
    'UNIVERSIDADE FEDERAL DE SÃO  JOÃO DEL REI - UFSJ' : 'UFSJ',
    'UNIVERSIDADE FEDERAL DE SÃO  JOÃO DEL-REI - UFSJ' : 'UFSJ',
    'UNIVERSIDADE FEDERAL DE SÃO JOÃO DEL REI' : 'UFSJ',
    'UNIVERSIDADE FEDERAL DE SÃO JOÃO DEL REI - UFSJ' : 'UFSJ',
    'UNIVERSIDADE FEDERAL DE SÃO JOÃO DEL-REI' : 'UFSJ',
    'UNIVERSIDADE FEDERAL DE SÃO JOÃO DEL-REI  UFSJ' : 'UFSJ',
    'UNIVERSIDADE FEDERAL DE SÃO JOÃO DEL-REI - UFSJ' : 'UFSJ',
    'UNIVERSIDADE FEDERAL DE SÃO PAULO' : 'UNIFESP',
    'UNIVERSIDADE FEDERAL DE SÃO PAULO - UNIFESP' : 'UNIFESP',
    'UNIVERSIDADE FEDERAL DE SERGIPE' : 'UFS',
    'UNIVERSIDADE FEDERAL DE UBERLANDIA' : 'UFU',
    'UNIVERSIDADE FEDERAL DE UBERLÂNDIA' : 'UFU',
    'UNIVERSIDADE FEDERAL DE UBERLANDIA - UFU' : 'UFU',
    'UNIVERSIDADE FEDERAL DE UBERLÂNDIA - UFU' : 'UFU',
    'UNIVERSIDADE FEDERAL DE UBERLÂNDIA – UFU' : 'UFU',
    'UNIVERSIDADE FEDERAL DE UBERLÂNDIA ? UFU' : 'UFU',
    'UNIVERSIDADE FEDERAL DE UBERLÂNDIA-UFU' : 'UFU',
    'UNIVERSIDADE FEDERAL DE VIÇOSA' : 'UFV',
    'UNIVERSIDADE FEDERAL DE VIÇOSA - UFV' : 'UFV',
    'UNIVERSIDADE FEDERAL DE VIÇOSA-UFV' : 'UFV',
    'UNIVERSIDADE FEDERAL DO ABC' : 'UFABC',
    'UNIVERSIDADE FEDERAL DO AGRESTE DE PERNAMBUCO' : 'UFAPE',
    'UNIVERSIDADE FEDERAL DO AMAPA' : 'UNIFAP',
    'UNIVERSIDADE FEDERAL DO AMAPÁ' : 'UNIFAP',
    'UNIVERSIDADE FEDERAL DO AMAZONAS' : 'UFAM',
    'UNIVERSIDADE FEDERAL DO CARIRI (UFCA)' : 'UFCA',
    'UNIVERSIDADE FEDERAL DO CEARA' : 'UFC',
    'UNIVERSIDADE FEDERAL DO CEARÁ' : 'UFC',
    'UNIVERSIDADE FEDERAL DO CEARÁ - UFC' : 'UFC',
    'UNIVERSIDADE FEDERAL DO CEARÁ – UFC' : 'UFC',
    'UNIVERSIDADE FEDERAL DO DELTA DO PARNAÍBA - UFDPAR' : 'UFDPAR',
    'UNIVERSIDADE FEDERAL DO ESPIRITO SANTO' : 'UFES',
    'UNIVERSIDADE FEDERAL DO ESPÍRITO SANTO' : 'UFES',
    'UNIVERSIDADE FEDERAL DO ESPIRÍTO SANTO - UFES' : 'UFES',
    'UNIVERSIDADE FEDERAL DO ESPÍRITO SANTO - UFES' : 'UFES',
    'UNIVERSIDADE FEDERAL DO ESTADO DO RIO DE JANEIRO' : 'UNIRIO',
    'UNIVERSIDADE FEDERAL DO ESTADO DO RIO DE JANEIRO - UFRJ' : 'UNIRIO',
    'UNIVERSIDADE FEDERAL DO MARANHÃO' : 'UFMA',
    'UNIVERSIDADE FEDERAL DO MARANHÃO-UFMA' : 'UFMA',
    'UNIVERSIDADE FEDERAL DO MATO GROSSO' : 'UFMT',
    'UNIVERSIDADE FEDERAL DO OESTE DO PARÁ' : 'UFOPA',
    'UNIVERSIDADE FEDERAL DO PAMPA' : 'UNIPAMPA',
    'UNIVERSIDADE FEDERAL DO PARÁ' : 'UFPA',
    'UNIVERSIDADE FEDERAL DO PARÁ - UFPA' : 'UFPA',
    'UNIVERSIDADE FEDERAL DO PARANA' : 'UFPR',
    'UNIVERSIDADE FEDERAL DO PARANÁ' : 'UFPR',
    'UNIVERSIDADE FEDERAL DO PARANÁ - UFPR' : 'UFPR',
    'UNIVERSIDADE FEDERAL DO PIAUÍ' : 'UFPI',
    'Universidade Federal do Piauí' : 'UFPI',
    'UNIVERSIDADE FEDERAL DO PIAUÍ  UFPI' : 'UFPI',
    'UNIVERSIDADE FEDERAL DO PIAUÍ - UFPI' : 'UFPI',
    'UNIVERSIDADE FEDERAL DO PIAUÍ- UFPI' : 'UFPI',
    'UNIVERSIDADE FEDERAL DO RECÔNCAVO DA BAHIA' : 'UFRB',
    'UNIVERSIDADE FEDERAL DO RECONCAVO DA BAHIA - UFRB' : 'UFRB',
    'UNIVERSIDADE FEDERAL DO RECÔNCAVO DA BAHIA - UFRB' : 'UFRB',
    'UNIVERSIDADE FEDERAL DO RIO DE JANEIRO' : 'UFRJ',
    'UNIVERSIDADE FEDERAL DO RIO DE JANEIRO - UFRJ' : 'UFRJ',
    'UNIVERSIDADE FEDERAL DO RIO DE JANEIRO ? UFRJ' : 'UFRJ',
    'UNIVERSIDADE FEDERAL DO RIO DE JANEIRO _ UFRJ' : 'UFRJ',
    'UNIVERSIDADE FEDERAL DO RIO DE JANEIRO -UFRJ' : 'UFRJ',
    'UNIVERSIDADE FEDERAL DO RIO GRANDE - FURG' : 'FURG',
    'UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE' : 'UFRN',
    'UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE - UFRN' : 'UFRN',
    'UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL' : 'UFRGS',
    'UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL - UFRGS' : 'UFRGS',
    'UNIVERSIDADE FEDERAL DO SUL DA BAHIA' : 'UFSB',
    'UNIVERSIDADE FEDERAL DO SUL DA BAHIA - UFSB' : 'UFSB',
    'UNIVERSIDADE FEDERAL DO SUL E SUDESTE DO PARÁ' : 'UNIFESSPA',
    'UNIVERSIDADE FEDERAL DO TOCANTINS' : 'UFT',
    'UNIVERSIDADE FEDERAL DO TOCANTINS - UFT' : 'UFT',
    'UNIVERSIDADE FEDERAL DO TRIÂNGULO MINEIRO' : 'UFTM',
    'UNIVERSIDADE FEDERAL DO TRIÂNGULO MINEIRO - UFTM' : 'UFTM',
    'UNIVERSIDADE FEDERAL DO TRIÂNGULO MINEIRO-UFTM' : 'UFTM',
    'UNIVERSIDADE FEDERAL DO VALE DO SÃO FRANCISCO' : 'UNIVASF',
    'UNIVERSIDADE FEDERAL DO VALE DO SÃO FRANCISCO - UNIVASF' : 'UNIVASF',
    'UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONHA E MUCURI' : 'UFVJM',
    'UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONHA E MUCURI - UFVJM' : 'UFVJM',
    'UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONHA E MUCURI- UFVJM' : 'UFVJM',
    'UNIVERSIDADE FEDERAL FLUMINENSE' : 'UFF',
    'UNIVERSIDADE FEDERAL FLUMINENSE - UFF' : 'UFF',
    'UNIVERSIDADE FEDERAL RURAL DA AMAZÔNIA' : 'UFRA',
    'UNIVERSIDADE FEDERAL RURAL DE PARNANBUCO - UFRPE' : 'UFRPE',
    'UNIVERSIDADE FEDERAL RURAL DE PERNAMBUCO' : 'UFRPE',
    'UNIVERSIDADE FEDERAL RURAL DE PERNAMBUCO - UFRPE' : 'UFRPE',
    'UNIVERSIDADE FEDERAL RURAL DE PERNAMBUCO (UFRPE)' : 'UFRPE',
    'UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO' : 'UFRRJ',
    'UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO-UFRRJ' : 'UFRRJ',
    'UNIVERSIDADE FEDERAL RURAL DO SEMI-ÁRIDO' : 'UFERSA',
    'UNIVERSIDADE FEDERAL RURAL DO SEMI-ARIDO - UFERSA' : 'UFERSA',
    'UNIVERSIDADE TECNÓLICA FEDERAL DO PARANÁ - UTFPR' : 'UTFPR',
    'UNIVERSIDADE TECNOLOGICA FEDERAL DO PARANA' : 'UTFPR',
    'UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ' : 'UTFPR',
    'UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ - UTFPR' : 'UTFPR',
    'UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ - UTFPR-PATO BRANCO' : 'UTFPR',
    'UTFPR - UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ' : 'UTFPR'
}

In [20]:
federais_melted = federais_melted.copy()
federais_melted['ifes_abrev'] = federais_melted['ifes_abrev'].map(correspondencias)


In [21]:
federais_melted.head()

,ID,ordem_respons,responsavel,parceria,ifes,ifes_abrev
2,ID00003,responsavel01,UNIVERSIDADE FEDERAL DO CEARÁ,Titular,Sim,UFC
24724,ID00011,responsavel02,UNIVERSIDADE FEDERAL DE PERNAMBUCO,Parceiro,Sim,UFPE
10,ID00011,responsavel01,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Titular,Sim,IFPE
14,ID00015,responsavel01,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Titular,Sim,IFGOIANO
23,ID00024,responsavel01,UNIVERSIDADE FEDERAL DE UBERLÂNDIA,Titular,Sim,UFU


In [22]:
data_parceiros_longo_completo = data.merge(federais_melted, on='ID', how='left')
data_parceiros_longo_completo['ifes'] = data_parceiros_longo_completo['ifes'].fillna('Não')
data_parceiros_longo_completo.head(12)

,ID,data_publicacao,diretoria,numero_revista,codigo,titulo,numero,data_deposito,data_prioridade,cod_prioridade,...,responsavel31,responsavel32,responsavel33,responsavel34,responsavel35,ordem_respons,responsavel,parceria,ifes,ifes_abrev
0,ID00001,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2013 016644-8,27/06/2013,NaN,NaN,...,None,None,None,None,None,NaN,NaN,NaN,Não,NaN
1,ID00002,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2014 001105-6,16/01/2014,18/01/2013,TW102201214,...,None,None,None,None,None,NaN,NaN,NaN,Não,NaN
2,ID00003,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 004929-9,13/03/2017,NaN,NaN,...,None,None,None,None,None,responsavel01,UNIVERSIDADE FEDERAL DO CEARÁ,Titular,Sim,UFC
3,ID00004,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 024659-0,17/11/2017,NaN,NaN,...,None,None,None,None,None,NaN,NaN,NaN,Não,NaN
4,ID00005,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 026178-6,05/12/2017,NaN,NaN,...,None,None,None,None,None,NaN,NaN,NaN,Não,NaN
5,ID00006,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 027316-4,06/12/2017,NaN,NaN,...,None,None,None,None,None,NaN,NaN,NaN,Não,NaN
6,ID00007,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 001913-9,29/01/2018,NaN,NaN,...,None,None,None,None,None,NaN,NaN,NaN,Não,NaN
7,ID00008,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 009789-0,15/05/2018,NaN,NaN,...,None,None,None,None,None,NaN,NaN,NaN,Não,NaN
8,ID00009,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 009796-2,15/05/2018,NaN,NaN,...,None,None,None,None,None,NaN,NaN,NaN,Não,NaN
9,ID00010,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 011938-9,13/06/2018,NaN,NaN,...,None,None,None,None,None,NaN,NaN,NaN,Não,NaN


In [23]:
data_parceiros_longo_ifes = data.merge(federais_melted, on='ID', how='right')
data_parceiros_longo_ifes.head(12)

,ID,data_publicacao,diretoria,numero_revista,codigo,titulo,numero,data_deposito,data_prioridade,cod_prioridade,...,responsavel31,responsavel32,responsavel33,responsavel34,responsavel35,ordem_respons,responsavel,parceria,ifes,ifes_abrev
0,ID00003,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 004929-9,13/03/2017,NaN,NaN,...,None,None,None,None,None,responsavel01,UNIVERSIDADE FEDERAL DO CEARÁ,Titular,Sim,UFC
1,ID00011,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 012005-0,13/06/2018,NaN,NaN,...,None,None,None,None,None,responsavel02,UNIVERSIDADE FEDERAL DE PERNAMBUCO,Parceiro,Sim,UFPE
2,ID00011,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 012005-0,13/06/2018,NaN,NaN,...,None,None,None,None,None,responsavel01,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Titular,Sim,IFPE
3,ID00015,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 012049-2,14/06/2018,NaN,NaN,...,None,None,None,None,None,responsavel01,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Titular,Sim,IFGOIANO
4,ID00024,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 012161-8,14/06/2018,NaN,NaN,...,None,None,None,None,None,responsavel01,UNIVERSIDADE FEDERAL DE UBERLÂNDIA,Titular,Sim,UFU
5,ID00026,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 012376-9,18/06/2018,NaN,NaN,...,None,None,None,None,None,responsavel01,UNIVERSIDADE FEDERAL DO MARANHÃO,Titular,Sim,UFMA
6,ID00030,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 012385-8,18/06/2018,NaN,NaN,...,None,None,None,None,None,responsavel01,UNIVERSIDADE TECNOLOGICA FEDERAL DO PARANA,Titular,Sim,UTFPR
7,ID00031,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 012394-7,18/06/2018,NaN,NaN,...,None,None,None,None,None,responsavel02,UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL - UFRGS,Parceiro,Sim,UFRGS
8,ID00035,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 012412-9,18/06/2018,NaN,NaN,...,None,None,None,None,None,responsavel01,UNIVERSIDADE FEDERAL DE CAMPINA GRANDE - PB,Titular,Sim,UFCG
9,ID00036,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 012435-8,18/06/2018,NaN,NaN,...,None,None,None,None,None,responsavel01,UNIVERSIDADE FEDERAL DE UBERLÂNDIA,Titular,Sim,UFU


## Exportacao em CSV e Excel

In [24]:
data_parceiros_longo_completo.to_csv("dados_parceiros_longo_completo.csv")
data_parceiros_longo_completo.to_excel("dados_parceiros_longo_completo.xlsx")
data_parceiros_longo_ifes.to_csv("dados_parceiros_longo_ifes.csv")
data_parceiros_longo_ifes.to_excel("dados_parceiros_longo_ifes.xlsx")